In [10]:
import pandas as pd
import datetime
import requests
import re
from bs4 import BeautifulSoup as bs
from dateutil.relativedelta import *
from sqlalchemy import create_engine
import os
from app import db
from models import Historical, Forecast, Station, Current

In [20]:
df.head()

,retrieval_time,drybulb,relative_humidity
datetime,,,
2019-06-24 06:00:00,2019-06-23 23:23:56.169981,16.850,85.0
2019-06-24 09:00:00,2019-06-23 23:23:56.174778,15.600,84.0
2019-06-24 12:00:00,2019-06-23 23:23:56.177488,18.150,81.0
2019-06-24 15:00:00,2019-06-23 23:23:56.180403,20.780,59.0
2019-06-24 18:00:00,2019-06-23 23:23:56.183046,19.408,71.0


In [19]:
df = get_forecasts()
for row in df.itertuples():
    print(row[0])


2019-06-24 06:00:00
2019-06-24 09:00:00
2019-06-24 12:00:00
2019-06-24 15:00:00
2019-06-24 18:00:00
2019-06-24 21:00:00
2019-06-25 00:00:00
2019-06-25 03:00:00
2019-06-25 06:00:00
2019-06-25 09:00:00
2019-06-25 12:00:00
2019-06-25 15:00:00
2019-06-25 18:00:00
2019-06-25 21:00:00
2019-06-26 00:00:00
2019-06-26 03:00:00
2019-06-26 06:00:00
2019-06-26 09:00:00
2019-06-26 12:00:00
2019-06-26 15:00:00
2019-06-26 18:00:00
2019-06-26 21:00:00
2019-06-27 00:00:00
2019-06-27 03:00:00
2019-06-27 06:00:00
2019-06-27 09:00:00
2019-06-27 12:00:00
2019-06-27 15:00:00
2019-06-27 18:00:00
2019-06-27 21:00:00
2019-06-28 00:00:00
2019-06-28 03:00:00
2019-06-28 06:00:00
2019-06-28 09:00:00
2019-06-28 12:00:00
2019-06-28 15:00:00
2019-06-28 18:00:00
2019-06-28 21:00:00
2019-06-29 00:00:00
2019-06-29 03:00:00


In [12]:
def get_forecasts():
    # 5-day 3-h forecast (only free one available)
    url = 'http://api.openweathermap.org/data/2.5/forecast?id=6167865&APPID=631d59f50ab1841ba7af0f0f706e1505'
    response = requests.get(url, verify=True)
    r = response.json()
    
    if response:
        df=pd.DataFrame()
        for dt in r['list']:    
            row={'datetime':dt['dt_txt'],
                 'retrieval_time':datetime.datetime.now(),
                 'drybulb':dt['main']['temp']-273.15,
                 'relative_humidity':dt['main']['humidity']}
                # note: Kelvin to Celcius
            
            df = df.append(row, ignore_index=True)
        df.set_index(pd.DatetimeIndex(df['datetime']), inplace=True)
        return df[['retrieval_time','drybulb', 'relative_humidity']]        

    else:
        print('Response Error')

In [3]:
def add_forecasts():
    session = db.session     # Creates a SQLAlchemy Session
        
    try:
        df = get_forecasts()
        for row in df.itertuples(): 
            data = Forecast(id=row[0], 
                            retrieval_time=row[1],
                            drybulb=row[2], 
                            relative_humidity=row[3])
            session.merge(data)
            session.commit()

    except:
        print('something messed up') 

In [ ]:
# df.to_csv('~/Desktop/test.csv')

In [ ]:
# Legacy
def add_scraped_forecasts():
    import time
    session = db.session     # Creates a SQLAlchemy Session
    tries = 0
   
    while tries <=10:
        try:
            df = scrape_forecasts()
            for row in df.itertuples(): 
                data = Forecast(id=row[0], drybulb=row[1], dewpoint=row[2])
                session.merge(data)
                session.commit()
            break

        except:
            print('can\'t get forecast')
            time.sleep(60)
            tries +=1